In [1]:
import time
from measure_time import measure_time

import pandas as pd
from ast import literal_eval #used for extracting comments (lists stored as strings)

import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import nltk
nltk.download("stopwords")
#--------#
from string import punctuation

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

def preprocess_text(text):
    words = re.split(r'[^а-яА-Я]', text) # разбиваем текст на слова
    tokens = list()
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS not in ['NPRO', 'PREP', 'CONJ', 'PRCL', 'INTJ', 'NUMR']:
            tokens.append(p.normal_form)
    tokens = [token for token in tokens if token not in russian_stopwords\
                and token != " " \
                and token.strip() not in punctuation]
    company_tokens_text_len = len(tokens)

    return " ".join(tokens), company_tokens_text_len

def preprocess_text_in_list(list_):
    prepr_list = []
    for comment in list_:
        prepr_comment = preprocess_text(comment)
        prepr_list.append(prepr_comment)
    return prepr_list

def unpack_comments(comments_list_str):
    if comments_list_str[-2:] != "']" and comments_list_str[-2:] != "\"]" and comments_list_str != "[]":
        last_ap_ind = comments_list_str.rfind("'")
        comments_list_str = comments_list_str[:last_ap_ind-2] + "]"
    return comments_list_str

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baltt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import os

groups_files = os.listdir('Collected')

groups_files

['Бонч_мемы_all_posts_31_07_2023.xlsx',
 'Бонч_основная_all_posts_31_07_2023.xlsx',
 'Бонч_подслушано_all_posts_31_07_2023.xlsx',
 'Бонч_поступление_all_posts_31_07_2023.xlsx',
 'ВШЭ_мемы_all_posts_31_07_2023.xlsx',
 'ВШЭ_основная_all_posts_31_07_2023.xlsx',
 'ВШЭ_подслушано.xlsx',
 'ВШЭ_поступление_all_posts_31_07_2023.xlsx',
 'ГУАП_мемы_all_posts_31_07_2023.xlsx',
 'ГУАП_осн_all_posts_31_07_2023.xlsx',
 'ГУАП_подслушано_all_posts_31_07_2023.xlsx',
 'ГУАП_поступ_all_posts_31_07_2023.xlsx',
 'ИТМО_мемы_all_posts_31_07_2023.xlsx',
 'ИТМО_основная_all_posts_31_07_2023.xlsx',
 'ИТМО_подслушано_all_posts_31_07_2023.xlsx',
 'ИТМО_поступление_all_posts_31_07_2023 — копия.xlsx',
 'ЛЭТИ_мемы_all_posts_31_07_2023.xlsx',
 'ЛЭТИ_основная_all_posts_31_07_2023.xlsx',
 'ЛЭТИ_подслушано_all_posts_31_07_2023.xlsx',
 'ЛЭТИ_постулпения_all_posts_31_07_2023.xlsx',
 'Мечникова_мемы_all_posts_31_07_2023.xlsx',
 'Мечникова_основная_all_posts_31_07_2023.xlsx',
 'Мечникова_подслушано_all_posts_31_07_2023.xlsx

In [3]:
import pandas as pd
from ast import literal_eval #used for extracting comments (lists stored as strings)

os.makedirs('Preprocessed', exist_ok = True)
for group in groups_files:
    st = time.time()
    print(f'Working on {group}')
    group_df = pd.read_excel(f'Collected/{group}')

    text_group_df = group_df[['Текст_поста', "Текст_комментариев"]]

    text_group_df_preprocessed = text_group_df['Текст_поста'].astype(str).apply(preprocess_text)
    text_group_df_preprocessed = pd.DataFrame(list(text_group_df_preprocessed), columns = ['text_tokens', 'text_length'])
    prepr_df = pd.concat([group_df, text_group_df_preprocessed], axis=1, join='inner')

    prepr_df['preprocessed_comments'] = text_group_df["Текст_комментариев"].apply(unpack_comments).apply(literal_eval).apply(preprocess_text_in_list)

    prepr_df.to_csv(f'Preprocessed/{group[:-5]}.csv')
    measure_time(st)

Working on Бонч_мемы_all_posts_31_07_2023.xlsx
Execution time: 0 hours 0 minutes 7 seconds
Working on Бонч_основная_all_posts_31_07_2023.xlsx
Execution time: 0 hours 5 minutes 8 seconds
Working on Бонч_подслушано_all_posts_31_07_2023.xlsx
Execution time: 0 hours 14 minutes 33 seconds
Working on Бонч_поступление_all_posts_31_07_2023.xlsx
Execution time: 0 hours 1 minutes 16 seconds
Working on ВШЭ_мемы_all_posts_31_07_2023.xlsx
Execution time: 0 hours 1 minutes 24 seconds
Working on ВШЭ_основная_all_posts_31_07_2023.xlsx
Execution time: 0 hours 4 minutes 57 seconds
Working on ВШЭ_подслушано.xlsx
Execution time: 0 hours 20 minutes 26 seconds
Working on ВШЭ_поступление_all_posts_31_07_2023.xlsx
Execution time: 0 hours 3 minutes 47 seconds
Working on ГУАП_мемы_all_posts_31_07_2023.xlsx
Execution time: 0 hours 0 minutes 7 seconds
Working on ГУАП_осн_all_posts_31_07_2023.xlsx
Execution time: 0 hours 2 minutes 15 seconds
Working on ГУАП_подслушано_all_posts_31_07_2023.xlsx
Execution time: 0 ho

In [12]:
prepr_groups_files = os.listdir('Preprocessed')

for group in prepr_groups_files:
    group_df = pd.read_csv(f'Preprocessed/{group}')
    group_df = group_df[['id_поста', 'text_tokens', 'text_length', 'preprocessed_comments']] #store only new comments to reduce memory consumption
    group_df.to_csv(f'Preprocessed/{group[:-4]}.csv')

In [14]:
group_df

,id_поста,Текст_поста,Дата_публикации_поста,Число_лайков,Число_комментариев,Число_просмотров,URL_photo,Текст_комментариев,text_tokens,text_length,preprocessed_comments
0,52701,Сохрани себе обязательно!\n\nВ статье ты найде...,2023-07-05 13:30:00,37,14,16682,-----,['🧡Контакты приемной комиссии ГИ: \n📞Телефон: ...,сохранить обязательно статья найти связаться п...,20,[('контакт приёмный комиссия ги телефон доб ко...
1,53005,Что такое контрактное обучение?📚\n\nЭто обучен...,2023-08-03 10:00:01,5,0,776,https://sun9-69.userapi.com/impg/nrTBJc6NNh8QV...,[],контрактный обучение обучение который студент ...,71,[]
2,52994,🚀 5 3 1 - это не отсчет до СТАРТа\n\nЭто пятьс...,2023-08-02 11:10:00,37,3,4531,https://sun9-16.userapi.com/impg/Ywej8r9IRnHX_...,"['Такой популярный университет, а мест на бюдж...",отсчёт старт счастливый политехник гладить бел...,98,[('популярный университет место бюджет очень д...
3,52992,"Будущие магистры, напоминание для вас🎓\n \n❗4 ...",2023-08-02 09:00:02,5,0,3166,https://sun9-4.userapi.com/impg/ybV72ZhMg5nY1t...,[],будущий магистр напоминание август заканчивать...,30,[]
4,52982,☘ Политех вошёл в топ-10 «зелёных» вузов Росси...,2023-08-01 14:01:20,129,0,7781,https://sun9-51.userapi.com/impg/ETubHW499FZwJ...,[],политех вош л топ зел ных вуз россия политех п...,54,[]
...,...,...,...,...,...,...,...,...,...,...,...
3210,10,"Что, где, когда: эксперт СПбПУ – о приемной ка...",2016-06-23 11:35:34,8,0,-----,-----,[],эксперт спбп приёмный кампания,4,[]
3211,8,Дорогие абитуриенты!\n\nВсе интересующие ответ...,2016-06-21 14:54:19,23,4,-----,https://sun9-58.userapi.com/impf/c630916/v6309...,"['О, молодцы! Реально круто!', 'Добрый день, а...",дорогой абитуриент интересовать ответ вопрос с...,103,"[('молодец реально круто', 3), ('добрый день в..."
3212,7,20 июня в Санкт-Петербургском политехническом ...,2016-06-20 11:32:29,19,0,-----,https://sun9-55.userapi.com/impf/c630916/v6309...,[],июнь санкт петербургский политехнический униве...,90,[]
3213,5,В Санкт-Петербургском политехническом универси...,2016-06-07 23:43:02,33,0,-----,-----,[],санкт петербургский политехнический университе...,79,[]
